In [1]:
import json
import time
import json
import string

import azure.cognitiveservices.speech as speechsdk

In [2]:
speech_region = "YOUR_AZURE_AI_SPEECH_REGION"
speech_key = "YOUR_AZURE_AI_SPEECH_KEY"

In [3]:
input_files = ['./wav/whatstheweatherlike.wav']
topic = ''
reference_text = 'What is the weather like?'
language = 'en-US'

In [4]:
def pronunciation_assessment_shortaudio(input_file, reference_text, language):

    # Setup audio configuration
    audio_config = speechsdk.audio.AudioConfig(filename=input_file)

    # Setup other speech configurations
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)

    ## Must disable miscue for the chatting scenario.
    json_string = {
        "GradingSystem": "HundredMark",
        "Granularity": "Phoneme",
        "EnableMiscue": False,
        "phonemeAlphabet": "IPA",
    }
    pronunciation_config = speechsdk.PronunciationAssessmentConfig(json_string=json.dumps(json_string))
    pronunciation_config.enable_prosody_assessment()
    # pronunciation_config.enable_content_assessment_with_topic(topic)
    pronunciation_config.reference_text = reference_text.strip()

    # Creates a speech recognizer using a file as audio input.
    speech_recognizer = speechsdk.SpeechRecognizer(
        speech_config=speech_config,
        language=language,
        audio_config=audio_config
    )
    # Apply pronunciation assessment config to speech recognizer
    pronunciation_config.apply_to(speech_recognizer)

    # Recognize using 'once'
    return speech_recognizer.recognize_once_async().get()

In [5]:
result = pronunciation_assessment_shortaudio(input_files[0], reference_text, language)
print(result.reason)

ResultReason.RecognizedSpeech


In [6]:
resultJson = json.loads(result.json)

print(resultJson['RecognitionStatus'])
print('Text: ' + resultJson['DisplayText'])
print(resultJson['NBest'][0]['PronunciationAssessment'])

Success
Text: What is the weather like?
{'AccuracyScore': 96.0, 'FluencyScore': 100.0, 'ProsodyScore': 90.6, 'CompletenessScore': 100.0, 'PronScore': 95.4}


In [7]:
print(json.dumps(resultJson, indent=4))

{
    "Id": "a9d144dce827439dab51824bbc81258f",
    "RecognitionStatus": "Success",
    "Offset": 500000,
    "Duration": 13700000,
    "Channel": 0,
    "DisplayText": "What is the weather like?",
    "SNR": 37.299374,
    "NBest": [
        {
            "Confidence": 0.963951,
            "Lexical": "What is the weather like",
            "ITN": "What is the weather like",
            "MaskedITN": "what is the weather like",
            "Display": "What is the weather like?",
            "PronunciationAssessment": {
                "AccuracyScore": 96.0,
                "FluencyScore": 100.0,
                "ProsodyScore": 90.6,
                "CompletenessScore": 100.0,
                "PronScore": 95.4
            },
            "Words": [
                {
                    "Word": "What",
                    "Offset": 500000,
                    "Duration": 3100000,
                    "PronunciationAssessment": {
                        "AccuracyScore": 100.0,
             

In [8]:
input_files = ['./wav/itsfinetoday.wav']
topic = ''
reference_text = '本日は晴天なり'
language = 'ja-JP'

In [9]:
result = pronunciation_assessment_shortaudio(input_files[0],reference_text, language)
print(result.reason)

ResultReason.RecognizedSpeech


In [10]:
resultJson = json.loads(result.json)

print(resultJson['RecognitionStatus'])
print('Text: ' + resultJson['DisplayText'])
print(resultJson['NBest'][0]['PronunciationAssessment'])

Success
Text: 本日は晴天なり。
{'AccuracyScore': 92.0, 'FluencyScore': 99.0, 'ProsodyScore': 90.4, 'CompletenessScore': 100.0, 'PronScore': 94.4}


In [11]:
print(json.dumps(resultJson, indent=4, ensure_ascii=False))

{
    "Id": "116fd8b030984164a50274d12172a289",
    "RecognitionStatus": "Success",
    "Offset": 1300000,
    "Duration": 19300000,
    "Channel": 0,
    "DisplayText": "本日は晴天なり。",
    "SNR": 45.51437,
    "NBest": [
        {
            "Confidence": 0.9355986,
            "Lexical": "本日 は 晴天 なり",
            "ITN": "本日 は 晴天 なり",
            "MaskedITN": "本日は晴天なり",
            "Display": "本日は晴天なり。",
            "PronunciationAssessment": {
                "AccuracyScore": 92.0,
                "FluencyScore": 99.0,
                "ProsodyScore": 90.4,
                "CompletenessScore": 100.0,
                "PronScore": 94.4
            },
            "Words": [
                {
                    "Word": "本日",
                    "Offset": 1300000,
                    "Duration": 6300000,
                    "PronunciationAssessment": {
                        "AccuracyScore": 88.0,
                        "ErrorType": "None",
                        "Feedback": {
           

In [12]:
def pronunciation_assessment_longaudio(input_file, reference_text, language):

    # Setup audio configuration
    audio_config = speechsdk.audio.AudioConfig(filename=input_file)

    # Setup other speech configurations
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)

    ## Must disable miscue for the chatting scenario.
    json_string = {
        "GradingSystem": "HundredMark",
        "Granularity": "Phoneme",
        "EnableMiscue": False,
        "phonemeAlphabet": "IPA",
    }
    pronunciation_config = speechsdk.PronunciationAssessmentConfig(json_string=json.dumps(json_string))
    pronunciation_config.enable_prosody_assessment()
    # pronunciation_config.enable_content_assessment_with_topic(topic)
    pronunciation_config.reference_text = reference_text.strip()

    # Creates a speech recognizer using a file as audio input.
    speech_recognizer = speechsdk.SpeechRecognizer(
        speech_config=speech_config,
        language=language,
        audio_config=audio_config
    )
    # Apply pronunciation assessment config to speech recognizer
    pronunciation_config.apply_to(speech_recognizer)

    # Prep for continuous recognition
    done = False
    assessment_result = []
    assessment_result_json = []

    def stop_cb(evt: speechsdk.SessionEventArgs):
        """callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        nonlocal done
        done = True

    def recognized(evt: speechsdk.SpeechRecognitionEventArgs):
        print("pronunciation assessment for: {}".format(evt.result.text))
        pronunciation_result = speechsdk.PronunciationAssessmentResult(evt.result)
        assessment_result.append(pronunciation_result)
        print("    Accuracy score: {}, prosody score: {}, pronunciation score: {}, completeness score : {}, fluency score: {}".format(
            pronunciation_result.accuracy_score, pronunciation_result.prosody_score, pronunciation_result.pronunciation_score,
            pronunciation_result.completeness_score, pronunciation_result.fluency_score
        ))

        result_json = evt.result.properties.get(speechsdk.PropertyId.SpeechServiceResponse_JsonResult)
        assessment_result_json.append(result_json)

    # Connect callbacks to the events fired by the speech recognizer
    speech_recognizer.recognized.connect(recognized)
    speech_recognizer.session_started.connect(lambda evt: print('SESSION STARTED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(lambda evt: print('SESSION STOPPED {}'.format(evt)))
    speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))

    # Stop continuous recognition on either session stopped or canceled events
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)

    # Start continuous pronunciation assessment
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)

    speech_recognizer.stop_continuous_recognition()

    return assessment_result, assessment_result_json    


In [13]:
input_files = ['./wav/season_fall.wav']
topic = ''
reference_text = 'The fall is a time of cozy contemplation for some, a sad time for others. No longer the far-flung ecstasy of summer and preceding the hibernation period of winter, it is a time of slowing down to reflect. It is no wonder the fall is called the spring of the philosopher. This season confronts us with what we have cultivated within. The ancients of many traditions honored this sacred time by taking autumns rich external symbolism as a tool for in a rebirth.'
language = 'en-US'

In [14]:
assessment_result, assessment_result_json = pronunciation_assessment_longaudio(input_files[0], reference_text, language)

SESSION STARTED: SessionEventArgs(session_id=26d0a3b44c434436980f4c08765603a9)
pronunciation assessment for: The fall is a time of cozy contemplation for some, a sad time for others.
    Accuracy score: 100.0, prosody score: 89.7, pronunciation score: 95.7, completeness score : 100.0, fluency score: 99.0
pronunciation assessment for: No longer the far-flung ecstasy of summer and preceding the hibernation period of winter, it is a time of slowing down to reflect.
    Accuracy score: 97.0, prosody score: 91.1, pronunciation score: 94.8, completeness score : 96.0, fluency score: 99.0
pronunciation assessment for: It is no wonder the fall is called the spring of the philosopher. This season confronts us with what we have cultivated within.
    Accuracy score: 93.0, prosody score: 88.4, pronunciation score: 92.8, completeness score : 96.0, fluency score: 98.0
pronunciation assessment for: The ancients of many traditions honored this sacred time by taking autumns rich external symbolism as a

In [15]:
assessment_result_values = vars(assessment_result[0])
print(assessment_result_values)

{'_accuracy_score': 100.0, '_pronunciation_score': 95.7, '_completeness_score': 100.0, '_fluency_score': 99.0, '_prosody_score': 89.7, '_content_assessment_result': None, '_words': [<azure.cognitiveservices.speech.PronunciationAssessmentWordResult object at 0x000001D12A673790>, <azure.cognitiveservices.speech.PronunciationAssessmentWordResult object at 0x000001D12A6735D0>, <azure.cognitiveservices.speech.PronunciationAssessmentWordResult object at 0x000001D12A651210>, <azure.cognitiveservices.speech.PronunciationAssessmentWordResult object at 0x000001D12A6522D0>, <azure.cognitiveservices.speech.PronunciationAssessmentWordResult object at 0x000001D12A653190>, <azure.cognitiveservices.speech.PronunciationAssessmentWordResult object at 0x000001D12A653250>, <azure.cognitiveservices.speech.PronunciationAssessmentWordResult object at 0x000001D12A652A10>, <azure.cognitiveservices.speech.PronunciationAssessmentWordResult object at 0x000001D12A652E90>, <azure.cognitiveservices.speech.Pronunciat

In [16]:
print(json.dumps(json.loads(assessment_result_json[0]), indent=4, ensure_ascii=False))

{
    "Id": "64a545926e32415393779b6ab2939c84",
    "RecognitionStatus": "Success",
    "Offset": 1100000,
    "Duration": 46800000,
    "Channel": 0,
    "DisplayText": "The fall is a time of cozy contemplation for some, a sad time for others.",
    "SNR": 40.46617,
    "NBest": [
        {
            "Confidence": 0.982249,
            "Lexical": "the fall is a time of cozy contemplation for some a sad time for others",
            "ITN": "the fall is a time of cozy contemplation for some a sad time for others",
            "MaskedITN": "the fall is a time of cozy contemplation for some a sad time for others",
            "Display": "The fall is a time of cozy contemplation for some, a sad time for others.",
            "PronunciationAssessment": {
                "AccuracyScore": 100.0,
                "FluencyScore": 99.0,
                "ProsodyScore": 89.7,
                "CompletenessScore": 100.0,
                "PronScore": 95.7
            },
            "Words": [
      

In [17]:
def calculate_long_audio_scores(reference_text, assessment_result, assessment_result_json):

    # Calculate the whole accuracy, prosody, fluency, and completeness scores
    recognized_words = []
    prosody_scores = []
    fluency_scores = []
    durations = []

    for each_result in assessment_result:
        recognized_words += each_result.words
        fluency_scores.append(each_result.fluency_score)
        if each_result.prosody_score is not None:
            prosody_scores.append(each_result.prosody_score)

    for each_result_json in assessment_result_json:
        jo = json.loads(each_result_json)
        nb = jo["NBest"][0]
        durations.append(sum([int(w["Duration"]) for w in nb["Words"]]))

    final_words = recognized_words

    # Calculate whole accuracy by averaging
    final_accuracy_scores = []
    for word in final_words:
        if word.error_type == 'Insertion':
            continue
        else:
            final_accuracy_scores.append(word.accuracy_score)
    accuracy_score = sum(final_accuracy_scores) / len(final_accuracy_scores)

    # Re-calculate the prosody score by averaging
    if len(prosody_scores) == 0:
        prosody_score = float("nan")
    else:
        prosody_score = sum(prosody_scores) / len(prosody_scores)

    # Re-calculate fluency score
    fluency_score = sum([x * y for (x, y) in zip(fluency_scores, durations)]) / sum(durations)

    # Calculate whole completeness score
    reference_words = [w.strip(string.punctuation) for w in reference_text.lower().split()]
    completeness_score = len([w for w in recognized_words if w.error_type == "None"]) / len(reference_words) * 100
    completeness_score = completeness_score if completeness_score <= 100 else 100

    print('    Paragraph accuracy score: {}, prosody score: {}, completeness score: {}, fluency score: {}'.format(
        accuracy_score, prosody_score, completeness_score, fluency_score
    ))

    for idx, word in enumerate(final_words):
        print('    {}: word: {}\taccuracy score: {}\terror type: {};'.format(
            idx + 1, word.word, word.accuracy_score, word.error_type
        ))

In [18]:
calculate_long_audio_scores(reference_text, assessment_result, assessment_result_json)

    Paragraph accuracy score: 97.03614457831326, prosody score: 90.00000000000001, completeness score: 97.59036144578313, fluency score: 98.75692182410424
    1: word: the	accuracy score: 100.0	error type: None;
    2: word: fall	accuracy score: 100.0	error type: None;
    3: word: is	accuracy score: 100.0	error type: None;
    4: word: a	accuracy score: 100.0	error type: None;
    5: word: time	accuracy score: 100.0	error type: None;
    6: word: of	accuracy score: 100.0	error type: None;
    7: word: cozy	accuracy score: 100.0	error type: None;
    8: word: contemplation	accuracy score: 96.0	error type: None;
    9: word: for	accuracy score: 100.0	error type: None;
    10: word: some	accuracy score: 100.0	error type: None;
    11: word: a	accuracy score: 100.0	error type: None;
    12: word: sad	accuracy score: 100.0	error type: None;
    13: word: time	accuracy score: 100.0	error type: None;
    14: word: for	accuracy score: 100.0	error type: None;
    15: word: others	accuracy scor

In [19]:
input_files = ['./wav/yukiguni.wav']
topic = ''
reference_text = '国境の長いトンネルを抜けると雪国であった。夜の底が白くなった。信号所に汽車が止まった。向側の座席から娘が立って来て、島村の前のガラス窓を落した。雪の冷気が流れこんだ。娘は窓いっぱいに乗り出して、遠くへ呼ぶように、「駅長さあん、駅長さあん」明りをさげてゆっくり雪を踏んで来た男は、襟巻で鼻の上まで包み、耳に帽子の毛皮を垂れていた。もうそんな寒さかと島村は外を眺めると、鉄道の官舎らしいバラックが山裾に寒々と散らばっているだけで、雪の色はそこまで行かぬうちに闇に呑まれていた。「駅長さん、私です、御機嫌よろしゅうございます」「ああ、葉子さんじゃないか。お帰りかい。また寒くなったよ」'
language = 'ja-JP'

In [20]:
assessment_result_ja, assessment_result_json_ja = pronunciation_assessment_longaudio(input_files[0], reference_text, language)

SESSION STARTED: SessionEventArgs(session_id=6a34f798a8584182bc5c0ba32c545fcf)
pronunciation assessment for: 国境の長いトンネルを抜けると雪国であった。
    Accuracy score: 94.0, prosody score: 97.3, pronunciation score: 97.1, completeness score : 100.0, fluency score: 100.0
pronunciation assessment for: 夜の底が白くなった。
    Accuracy score: 95.0, prosody score: 89.2, pronunciation score: 94.7, completeness score : 100.0, fluency score: 100.0
pronunciation assessment for: 信号所に汽車が止まった。
    Accuracy score: 96.0, prosody score: 88.9, pronunciation score: 94.8, completeness score : 100.0, fluency score: 100.0
pronunciation assessment for: 向側の座席から娘が立って来て、島村の前のガラス窓を落した。
    Accuracy score: 95.0, prosody score: 91.9, pronunciation score: 94.8, completeness score : 100.0, fluency score: 95.0
pronunciation assessment for: 雪の冷気が流れこんだ。
    Accuracy score: 97.0, prosody score: 0.0, pronunciation score: 59.4, completeness score : 100.0, fluency score: 100.0
pronunciation assessment for: 娘は窓いっぱいに乗り出して遠くへ呼ぶように。
    Accuracy scor

In [21]:
calculate_long_audio_scores(reference_text, assessment_result_ja, assessment_result_json_ja)

    Paragraph accuracy score: 93.79012345679013, prosody score: 77.2, completeness score: 100, fluency score: 95.15123301459487
    1: word: 国境	accuracy score: 94.0	error type: None;
    2: word: の	accuracy score: 98.0	error type: None;
    3: word: 長い	accuracy score: 82.0	error type: None;
    4: word: トンネル	accuracy score: 86.0	error type: None;
    5: word: を	accuracy score: 96.0	error type: None;
    6: word: 抜ける	accuracy score: 88.0	error type: None;
    7: word: と	accuracy score: 100.0	error type: None;
    8: word: 雪国	accuracy score: 90.0	error type: None;
    9: word: で	accuracy score: 100.0	error type: None;
    10: word: あっ	accuracy score: 100.0	error type: None;
    11: word: た	accuracy score: 100.0	error type: None;
    12: word: 夜	accuracy score: 96.0	error type: None;
    13: word: の	accuracy score: 100.0	error type: None;
    14: word: 底	accuracy score: 72.0	error type: None;
    15: word: が	accuracy score: 98.0	error type: None;
    16: word: 白く	accuracy score: 98.0	erro